# Original Data

Mobile operators have historical records on which customers ultimately ended up churning and which continued using the service. We can use this historical information to construct an ML model of one mobile operator’s churn using a process called training. After training the model, we can pass the profile information of an arbitrary customer (the same profile information that we used to train the model) to the model, and have the model predict whether this customer is going to churn. Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

```sh
wget http://dataminingconsultant.com/DKD2e_data_sets.zip
unzip -o DKD2e_data_sets.zip
```

In [1]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
spark.read.format('csv').options(header='true', inferSchema='true').load('data/churn.txt').show()

+-----+--------------+---------+--------+----------+----------+-------------+--------+---------+----------+--------+---------+----------+----------+-----------+------------+---------+----------+-----------+--------------+------+
|State|Account Length|Area Code|   Phone|Int'l Plan|VMail Plan|VMail Message|Day Mins|Day Calls|Day Charge|Eve Mins|Eve Calls|Eve Charge|Night Mins|Night Calls|Night Charge|Intl Mins|Intl Calls|Intl Charge|CustServ Calls|Churn?|
+-----+--------------+---------+--------+----------+----------+-------------+--------+---------+----------+--------+---------+----------+----------+-----------+------------+---------+----------+-----------+--------------+------+
|   KS|           128|      415|382-4657|        no|       yes|           25|   265.1|      110|     45.07|   197.4|       99|     16.78|     244.7|         91|       11.01|     10.0|         3|        2.7|             1|False.|
|   OH|           107|      415|371-7191|        no|       yes|           26|   161.

# [ORACLE1](https://console.aws.amazon.com/rds/home?region=us-east-1#database:id=oracle1-db)

![Transactions](img/transactions_erd.PNG)

In [18]:
%%sql -c oracle1
SELECT * FROM OT.TRANSACTION_TYPES

,ID,DESCRIPTION
0,1.000000000000000000,Serviços
1,2.000000000000000000,Educação
2,3.000000000000000000,Transporte
3,4.000000000000000000,Restaurante
4,5.000000000000000000,Lazer
5,6.000000000000000000,Supermercado
6,7.000000000000000000,Outros


## ADHOC

[![consulta](img/adhoc.PNG)](https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:229343956935:execution:DoraImportMachine:20200415160100992055.didone.ORACLE1)

In [19]:
%%sql -v
SELECT t.USER_ID 
     , t.TRANSACTION_DATE 
     , t.TRANSACTION_TYPE 
     , t.VALUE 
     , tt.DESCRIPTION 
  FROM ORACLE1.OT.TRANSACTIONS t
  JOIN ORACLE1.OT.TRANSACTION_TYPES tt ON t.CATEGORY_ID = tt.ID 
 ORDER BY t.USER_ID
 LIMIT 10

INFO:ORACLE1.OT.TRANSACTIONS is updated: 2020-04-15 (7 days)
INFO:ORACLE1.OT.TRANSACTION_TYPES is updated: 2020-04-15 (7 days)
INFO:Execution Time: 1.827333


,USER_ID,TRANSACTION_DATE,TRANSACTION_TYPE,VALUE,DESCRIPTION
0,1.000000000000000000,2020-03-04 19:20:22,DEBITO,253.0800000000,Outros
1,1.000000000000000000,2020-03-29 21:51:10,CREDITO,413.1600000000,Educação
2,1.000000000000000000,2020-03-10 05:51:52,DEBITO,570.5300000000,Outros
3,1.000000000000000000,2020-03-30 02:05:05,DEBITO,721.3500000000,Serviços
4,1.000000000000000000,2020-03-25 06:48:34,CREDITO,287.6800000000,Outros
5,22.000000000000000000,2020-03-20 10:52:58,CREDITO,62.7600000000,Outros
6,23.000000000000000000,2020-03-01 13:34:02,CREDITO,280.2600000000,Serviços
7,23.000000000000000000,2020-03-23 01:28:24,CREDITO,773.8600000000,Transporte
8,23.000000000000000000,2020-03-14 12:38:09,CREDITO,258.4300000000,Educação
9,23.000000000000000000,2020-03-12 16:22:23,CREDITO,38.0300000000,Transporte


In [20]:
%%sql
CREATE OR REPLACE VIEW CATEGORIES AS
SELECT t.USER_ID `USER_ID`
     , ts.total `T_SERVICOS`
     , td.total `T_EDUCACAO`
     , tr.total `T_RESTAURANTE`
     , tt.total `T_TRANSPORTE`
     , tl.total `T_LAZER`
     , tm.total `T_SUPERMERCADO`
     , to.total `T_OUTROS`
  FROM (
SELECT distinct(USER_ID) `USER_ID`
  FROM ORACLE1.OT.TRANSACTIONS) t
  LEFT OUTER JOIN (
SELECT USER_ID
     , SUM(VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS
 WHERE CATEGORY_ID = 1
 GROUP BY USER_ID) ts ON ts.USER_ID = t.USER_ID
  LEFT OUTER JOIN (
SELECT USER_ID
     , SUM(VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS
 WHERE CATEGORY_ID = 2
 GROUP BY USER_ID) td ON td.USER_ID = t.USER_ID
  LEFT OUTER JOIN (
SELECT USER_ID
     , SUM(VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS
 WHERE CATEGORY_ID = 3
 GROUP BY USER_ID) tr ON tr.USER_ID = t.USER_ID
  LEFT OUTER JOIN (
SELECT USER_ID
     , SUM(VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS
 WHERE CATEGORY_ID = 4
 GROUP BY USER_ID) tt ON tt.USER_ID = t.USER_ID
  LEFT OUTER JOIN (
SELECT USER_ID
     , SUM(VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS
 WHERE CATEGORY_ID = 5
 GROUP BY USER_ID) tl ON tl.USER_ID = t.USER_ID
  LEFT OUTER JOIN (
SELECT USER_ID
     , SUM(VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS
 WHERE CATEGORY_ID = 6
 GROUP BY USER_ID) tm ON tm.USER_ID = t.USER_ID
  LEFT OUTER JOIN (
SELECT USER_ID
     , SUM(VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS
 WHERE CATEGORY_ID = 7
 GROUP BY USER_ID) to ON to.USER_ID = t.USER_ID

""


In [21]:
%%sql
SELECT * 
  FROM CATEGORIES 
 LIMIT 10

,USER_ID,T_SERVICOS,T_EDUCACAO,T_RESTAURANTE,T_TRANSPORTE,T_LAZER,T_SUPERMERCADO,T_OUTROS
0,1.000000000000000000,721.3500000000,413.1600000000,None,None,None,None,1111.2900000000
1,22.000000000000000000,None,None,None,None,None,None,62.7600000000
2,23.000000000000000000,711.9000000000,258.4300000000,1069.8300000000,20.3100000000,None,189.7400000000,None
3,24.000000000000000000,247.6500000000,0.9000000000,599.5800000000,None,101.6900000000,None,938.8000000000
4,25.000000000000000000,632.9000000000,384.4800000000,843.8300000000,1001.4000000000,146.4800000000,None,590.6700000000
5,26.000000000000000000,None,360.0200000000,None,None,308.2500000000,None,None
6,27.000000000000000000,None,None,None,None,444.6500000000,None,None
7,28.000000000000000000,65.9200000000,None,622.8500000000,None,None,None,42.8800000000
8,29.000000000000000000,63.9900000000,285.9000000000,None,251.8300000000,None,172.0300000000,440.0200000000
9,30.000000000000000000,374.0400000000,None,None,None,None,None,None


# FIN_BR

![Transactions](img/users_erd.PNG)

In [6]:
%%sql
select * from fin_br.fin.ADDRESS limit 2

,uuid,street_name,street_number,city,state,country,postcode
0,1,Rua Rio de Janeiro,2876,Cariacica,Distrito Federal,Brazil,55677
1,3,Rua Pernambuco,6665,Maranguape,Amazonas,Brazil,43369


In [7]:
%%sql
select * from csv.`workspace://churn/data/estados.csv.gz`

,_c0,_c1
0,Acre,AC
1,Alagoas,AL
2,Amapá,AP
3,Amazonas,AM
4,Bahia,BA
5,Ceará,CE
6,Distrito Federal,DF
7,Espírito Santo,ES
8,Goiás,GO
9,Maranhão,MA


In [8]:
%%sql
SELECT *
 FROM fin_br.fin.ADDRESS a
  JOIN csv.`workspace://churn/data/estados.csv.gz` e ON upper(e._c0) = upper(a.state)
LIMIT 10

,uuid,street_name,street_number,city,state,country,postcode,_c0,_c1
0,1,Rua Rio de Janeiro,2876,Cariacica,Distrito Federal,Brazil,55677,Distrito Federal,DF
1,3,Rua Pernambuco,6665,Maranguape,Amazonas,Brazil,43369,Amazonas,AM
2,6,Rua Maranhão,1500,Queimados,Rondônia,Brazil,46752,Rondônia,RO
3,7,Avenida da Legalidade,4754,São Paulo,Pernambuco,Brazil,82969,Pernambuco,PE
4,9,Rua Sete de Setembro,1782,Maceió,Maranhão,Brazil,25541,Maranhão,MA
5,10,Rua Paraíba,7290,Aracaju,Mato Grosso,Brazil,68194,Mato Grosso,MT
6,11,Rua São Paulo,8446,São José do Rio Preto,Mato Grosso do Sul,Brazil,77449,Mato Grosso do Sul,MS
7,13,Rua Rui Barbosa,1932,Americana,Bahia,Brazil,12315,Bahia,BA
8,20,Rua Vinte E Um,6205,Sinop,Roraima,Brazil,17814,Roraima,RR
9,21,Rua Rui Barbosa,3366,Porto Velho,Amazonas,Brazil,62035,Amazonas,AM


In [14]:
%%sql
CREATE TABLE CHURN AS
SELECT a.postcode `CEP`
     , e._c1 `ESTADO`
     , u.gender `GENERO`
     , substring(u.cell,2,2) `DDD`
     , substring(replace(u.cell,'-',''),6) `CELULAR`
     , datediff(now(), u.dob)/365 `IDADE`
     , datediff(now(), l.registered)/365 `IDADE_CONTA`
     , cast(nvl(c.total, 0) as float) `CREDITO`
     , cast(nvl(d.total, 0) as float) `DEBITO`
     , cast(nvl(cat.T_SERVICOS, 0) AS float) `T_SERVICOS`
     , cast(nvl(cat.T_EDUCACAO, 0) AS float) `T_EDUCACAO`
     , cast(nvl(cat.T_RESTAURANTE, 0) AS float) `T_RESTAURANTE`
     , cast(nvl(cat.T_TRANSPORTE, 0) AS float) `T_TRANSPORTE`
     , cast(nvl(cat.T_LAZER, 0) AS float) `T_LAZER`
     , cast(nvl(cat.T_SUPERMERCADO, 0) AS float) `T_SUPERMERCADO`
     , cast(nvl(cat.T_OUTROS, 0) AS float) `T_OUTROS`
     , if(l.inactivate_date is null, 'False.', 'True.') `CHURN`
  FROM fin_br.fin.LOGIN l 
  JOIN fin_br.fin.USERS u ON l.UUID = u.UUID 
  JOIN fin_br.fin.ADDRESS a ON a.UUID = u.UUID 
  JOIN csv.`workspace://churn/data/estados.csv.gz` e ON upper(e._c0) = upper(a.state)
  LEFT OUTER JOIN (
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.TRANSACTION_TYPE = 'CREDITO'
 GROUP BY t.USER_ID) c ON c.USER_ID = l.UUID
  LEFT OUTER JOIN (
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.TRANSACTION_TYPE = 'DEBITO'
 GROUP BY t.USER_ID) d ON d.USER_ID = l.UUID
  LEFT OUTER JOIN CATEGORIES cat ON cat.USER_ID = l.UUID

""


In [15]:
%%sql
show tables

,database,tableName,isTemporary
0,dora_didone,categories,False
1,dora_didone,churn,False
2,dora_didone,t_servicos,False


In [22]:
%%sql
select * from CHURN limit 10

,CEP,ESTADO,GENERO,DDD,CELULAR,IDADE,IDADE_CONTA,CREDITO,DEBITO,T_SERVICOS,T_EDUCACAO,T_RESTAURANTE,T_TRANSPORTE,T_LAZER,T_SUPERMERCADO,T_OUTROS,CHURN
0,55677,DF,male,47,50211417,53.775342,2.290411,700.840027,1544.959961,721.349976,413.160004,0.0,0.0,0.0,0.0,1111.290039,False.
1,43369,AM,male,46,16013498,74.328767,1.369863,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
2,83328,MG,female,13,35325396,27.331507,8.852055,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
3,46752,RO,male,15,95345287,64.942466,2.410959,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
4,82969,PE,male,37,28499799,67.339726,13.526027,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
5,83486,PI,female,91,11044136,25.904110,16.265753,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
6,25541,MA,female,40,58637152,67.704110,6.805479,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
7,68194,MT,male,37,04490210,69.169863,12.723288,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
8,77449,MS,female,22,34665989,65.873973,5.054795,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
9,58717,BA,female,69,09792455,39.347945,7.504110,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False.
